In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
df = pd.read_csv('netflix1.csv')

df.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,United States,9/25/2021,2020,PG-13,90 min,Documentaries
1,s3,TV Show,Ganglands,Julien Leclercq,France,9/24/2021,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
2,s6,TV Show,Midnight Mass,Mike Flanagan,United States,9/24/2021,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries"
3,s14,Movie,Confessions of an Invisible Girl,Bruno Garotti,Brazil,9/22/2021,2021,TV-PG,91 min,"Children & Family Movies, Comedies"
4,s8,Movie,Sankofa,Haile Gerima,United States,9/24/2021,1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies"


In [5]:
df.shape

(8790, 10)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8790 non-null   object
 1   type          8790 non-null   object
 2   title         8790 non-null   object
 3   director      8790 non-null   object
 4   country       8790 non-null   object
 5   date_added    8790 non-null   object
 6   release_year  8790 non-null   int64 
 7   rating        8790 non-null   object
 8   duration      8790 non-null   object
 9   listed_in     8790 non-null   object
dtypes: int64(1), object(9)
memory usage: 686.8+ KB


In [7]:
df.describe()

,release_year
count,8790.000000
mean,2014.183163
std,8.825466
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [8]:
df.duplicated().value_counts()

False    8790
dtype: int64

In [ ]:
#No duplicated values and no Null.

In [12]:
#date_added is object and needs to be a datatime object not a string
#Format here is month day year

df['date_added'] = pd.to_datetime(df['date_added'], format='%m/%d/%Y')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8790 entries, 0 to 8789
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       8790 non-null   object        
 1   type          8790 non-null   object        
 2   title         8790 non-null   object        
 3   director      8790 non-null   object        
 4   country       8790 non-null   object        
 5   date_added    8790 non-null   datetime64[ns]
 6   release_year  8790 non-null   int64         
 7   rating        8790 non-null   object        
 8   duration      8790 non-null   object        
 9   listed_in     8790 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 686.8+ KB


In [25]:
#Listed in should be seperated. must split the columns
#as should director
#I'll make the assumption that the first in the list of comma seperated values is the primary classifcation (Do not have the domain context)

# Split the 'Listed_in' column by commas and expand it into separate columns
df[['Listed_in_1', 'Listed_in_2', 'Listed_in_3']] = df['listed_in'].str.split(',', expand=True)


df[['director_1','director_others']] = df['director'].str.split(',', 1,expand=True)

<ipython-input-25-c858dd993795>:9: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['director_1','director_others']] = df['director'].str.split(',', 1,expand=True)


In [27]:
df['duration'].value_counts()

1 Season      1791
2 Seasons      421
3 Seasons      198
90 min         152
97 min         146
              ... 
5 min            1
16 min           1
186 min          1
193 min          1
11 Seasons       1
Name: duration, Length: 220, dtype: int64

In [28]:
#drop column I won't need:
#'show_id'
#'listed_in'
#'director'

columns_to_drop = ['show_id','listed_in','director']
df = df.drop(columns=columns_to_drop)

In [29]:
#duration is the wrong data type.
#sometimes its 1/2 season other times its 90 'mins' etc..
#The issue is here that movies and tv series are in the same dataset.. we could split this out into two datasets and change to an interger value or..
#keep as the same dataset and create a new collumn to convert duration to a consistent unit and make the assumption that 1 series is x amount of episodes..

In [30]:
#In this example, I assumed that 1 season is equivalent to 30 episodes and 1 episode is 24 minutes.

df['duration_numeric'] = df['duration'].str.extract('(\d+)').astype(float)

# Convert durations to minutes based on their units
df.loc[df['duration'].str.contains('Season'), 'duration_numeric'] *= 30 * 24 * 60  # Assuming 1 season is 30 episodes, 1 episode is 24 minutes
df.loc[df['duration'].str.contains('min'), 'duration_numeric']  # No conversion needed for 'min'

# Drop the original 'duration' column
df = df.drop(columns=['duration'])

df.head()


,type,title,country,date_added,release_year,rating,Listed_in_1,Listed_in_2,Listed_in_3,director_1,director_others,duration_numeric
0,Movie,Dick Johnson Is Dead,United States,9/25/2021,2020,PG-13,Documentaries,None,None,Kirsten Johnson,None,90.0
1,TV Show,Ganglands,France,9/24/2021,2021,TV-MA,Crime TV Shows,International TV Shows,TV Action & Adventure,Julien Leclercq,None,43200.0
2,TV Show,Midnight Mass,United States,9/24/2021,2021,TV-MA,TV Dramas,TV Horror,TV Mysteries,Mike Flanagan,None,43200.0
3,Movie,Confessions of an Invisible Girl,Brazil,9/22/2021,2021,TV-PG,Children & Family Movies,Comedies,None,Bruno Garotti,None,91.0
4,Movie,Sankofa,United States,9/24/2021,1993,TV-MA,Dramas,Independent Movies,International Movies,Haile Gerima,None,125.0


In [ ]:
all the best, and good luck!